In [14]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [12]:
t = torch.Tensor([
    [1],
    [2],
    [3]
])
t

tensor([[1.],
        [2.],
        [3.]])

In [13]:
t.int()

tensor([[1],
        [2],
        [3]], dtype=torch.int32)

In [3]:
t.size()

torch.Size([3, 1])

In [4]:
t.squeeze(1)

tensor([1., 2., 3.])

In [5]:
t.to(torch.float)

tensor([[1.],
        [2.],
        [3.]])

In [6]:
t

tensor([[1.],
        [2.],
        [3.]])

In [8]:
d = {
    'a': np.zeros((2, 3)),
    'a': np.zeros((3, 4)),
    'a': np.zeros((5, 6)),
}
d

{'a': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])}

In [9]:
df = pd.DataFrame(d)
df.head()

ValueError: If using all scalar values, you must pass an index

In [10]:
??pd.DataFrame

In [15]:
cm = confusion_matrix([1, 2, 3, 4, 3, 2, 1], [1, 3, 2, 4, 3, 5, 3])

In [16]:
cm

array([[1, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0]])

In [17]:
df = pd.DataFrame(cm)
df.head()

,0,1,2,3,4
0,1,0,1,0,0
1,0,0,1,0,1
2,0,1,1,0,0
3,0,0,0,1,0
4,0,0,0,0,0


In [19]:
pd.concat([df], keys=['label'], names=['pred'])

0  1  2  3  4
pred                  
label 0  1  0  1  0  0
      1  0  0  1  0  1
      2  0  1  1  0  0
      3  0  0  0  1  0
      4  0  0  0  0  0